In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import hashlib
import random

from tqdm import tqdm
import rdkit.Chem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import glob
import torch
torch.set_num_threads(1)
torch.multiprocessing.set_sharing_strategy('file_system')

%matplotlib inline

In [2]:
import sys
tankbind_src_folder = "/gxr/luwei/tankbind_github/TankBind/tankbind"
sys.path.insert(0, tankbind_src_folder)

import logging

from data import TankBindDataSet
from torch_geometric.loader import DataLoader
from model import get_model

In [3]:
result_folder = "/gxr/luwei/tankbind_github/predictions/"
os.system(f"mkdir -p {result_folder}")

rdkit_folder = f"{result_folder}/rdkit/"
os.system(f"mkdir -p {rdkit_folder}")

0

In [4]:
from feature_utils import read_mol, generate_sdf_from_smiles_using_rdkit, extract_torchdrug_feature_from_mol

compound_dict = {}
pre = "/gxr/luwei/pdbbind/pdbbind2020"
test = np.loadtxt("/gxr/luwei/packages/EquiBind/data/timesplit_test", dtype=str)
for name in tqdm(test):
    mol, _ = read_mol(f"{pre}/renumber_atom_index_same_as_smiles/{name}.sdf", None)
    smiles = Chem.MolToSmiles(mol)

    rdkit_mol_path = f"{rdkit_folder}/{name}_ligand.sdf"
    generate_sdf_from_smiles_using_rdkit(smiles, rdkit_mol_path, shift_dis=0)

    mol, _ = read_mol(rdkit_mol_path, None)
    compound_dict[name] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
torch.save(compound_dict, f"{result_folder}/pdbbind_test_compound_dict_based_on_rdkit.pt")

 87%|████████▋ | 316/363 [02:13<00:12,  3.68it/s]RDKit ERROR: [23:28:41] UFFTYPER: Unrecognized hybridization for atom: 17
RDKit ERROR: [23:28:41] UFFTYPER: Unrecognized atom type: Fe (17)
/gxr/luwei/anaconda3/envs/py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
100%|██████████| 363/363 [02:33<00:00,  2.37it/s]


In [5]:
dataset = TankBindDataSet("/gxr/luwei/pdbbind/pdbbind2020/test_dataset", 
                          proteinMode=0, compoundMode=1, pocket_radius=20, predDis=True)
dataset.compound_dict = torch.load(f"{result_folder}/pdbbind_test_compound_dict_based_on_rdkit.pt")
# dataset.data = dataset.data.query("not use_compound_com").reset_index(drop=True)
data_loader = DataLoader(dataset, batch_size=1, 
                         follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=8, pin_memory=True)


['/gxr/luwei/pdbbind/pdbbind2020/test_dataset/processed/data.pt', '/gxr/luwei/pdbbind/pdbbind2020/test_dataset/processed/protein.pt', '/gxr/luwei/pdbbind/pdbbind2020/test_dataset/processed/compound.pt']


In [6]:
# device = 'cpu'
device = "cuda:2"
logging.basicConfig(level=logging.INFO)
model = get_model(0, logging, device)
model.eval()
model.load_state_dict(torch.load(f"{tankbind_src_folder}/../saved_models/self_dock.pt"))

affinity_pred_list = []
y_pred_list = []
for data in tqdm(data_loader):
    data = data.to(device)
    with torch.no_grad():
        y_pred, affinity_pred = model(data)
    affinity_pred_list.append(affinity_pred.detach().cpu())
    for i in range(data.y_batch.max() + 1):
        y_pred_list.append((y_pred[data['y_batch'] == i]).detach().cpu())

affinity_pred_list = torch.cat(affinity_pred_list)

23:29:01   5 stack, readout2, pred dis map add self attention and GVP embed, compound model GIN


100%|██████████| 2879/2879 [01:35<00:00, 30.06it/s]


In [7]:
output_info_chosen = dataset.data
output_info_chosen['affinity'] = affinity_pred_list
output_info_chosen['dataset_index'] = range(len(output_info_chosen))
output_info_chosen = output_info_chosen.query("not use_compound_com").reset_index(drop=True)

chosen = output_info_chosen.loc[output_info_chosen.groupby(['protein_name', 'compound_name'], 
                                                           sort=False)['affinity'].agg('idxmax')].reset_index()

In [8]:
from generation_utils import get_LAS_distance_constraint_mask, get_info_pred_distance, write_with_new_coords

device = "cpu"
for idx, line in tqdm(chosen.iterrows(), total=chosen.shape[0]):
    name = compound_name = line['compound_name']
    dataset_index = line['dataset_index']
    coords = dataset[dataset_index].coords.to(device)
    protein_nodes_xyz = dataset[dataset_index].node_xyz.to(device)
    n_compound = coords.shape[0]
    n_protein = protein_nodes_xyz.shape[0]
    y_pred = y_pred_list[dataset_index].reshape(n_protein, n_compound).to(device)
    y = dataset[dataset_index].dis_map.reshape(n_protein, n_compound).to(device)
    compound_pair_dis_constraint = torch.cdist(coords, coords)
    rdkit_mol_path = f"{rdkit_folder}/{name}_ligand.sdf"
    # mol = Chem.MolFromMolFile(rdkit_mol_path)
    mol, _ = read_mol(rdkit_mol_path, None)
    LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()
    pred_dist_info = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint,
                                  LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                                  n_repeat=1, show_progress=False)

    toFile = f'{result_folder}/{name}_tankbind_chosen.sdf'
    new_coords = pred_dist_info.sort_values("loss")['coords'].iloc[0].astype(np.double)
    write_with_new_coords(mol, new_coords, toFile)


100%|██████████| 363/363 [24:05<00:00,  3.98s/it]


In [9]:
# taken from https://github.com/nghiaho12/rigid_transform_3D/blob/master/rigid_transform_3D.py
# "Least-Squares Fitting of Two 3-D Point Sets", Arun, K. S. and Huang, T. S. and Blostein, S. D, IEEE Transactions on Pattern Analysis and Machine Intelligence, Volume 9 Issue 5, May 1987
# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B, correct_reflection=True):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0 and correct_reflection:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

def compute_RMSD(a, b):
    # correct rmsd calculation.
    return np.sqrt((((a-b)**2).sum(axis=-1)).mean())

def kabsch_RMSD(new_coords, coords):
    out = new_coords.T
    target = coords.T
    ret_R, ret_t = rigid_transform_3D(out, target, correct_reflection=False)
    out = (ret_R@out) + ret_t
    return compute_RMSD(target.T, out.T)

In [10]:
info = []
pre = "/gxr/luwei/pdbbind/pdbbind2020"
test = np.loadtxt("/gxr/luwei/packages/EquiBind/data/timesplit_test", dtype=str)
for pdb in test:
    mol, _ = read_mol(f"{pre}/renumber_atom_index_same_as_smiles/{pdb}.sdf", None)
    mol_pred, _ = read_mol(f"{result_folder}/{pdb}_tankbind_chosen.sdf", None)
    # mol = Chem.MolFromMolFile(f"{pre}/renumber_atom_index_same_as_smiles/{pdb}.sdf")
    # mol_pred = Chem.MolFromMolFile(f"{result_folder}/{pdb}_tankbind_chosen.sdf")

    sm = Chem.MolToSmiles(mol)
    m_order = list(mol.GetPropsAsDict(includePrivate=True, includeComputed=True)['_smilesAtomOutputOrder'])
    mol = Chem.RenumberAtoms(mol, m_order)
    mol = Chem.RemoveHs(mol)
    true_ligand_pos = np.array(mol.GetConformer().GetPositions())

    sm = Chem.MolToSmiles(mol_pred)
    m_order = list(mol_pred.GetPropsAsDict(includePrivate=True, includeComputed=True)['_smilesAtomOutputOrder'])
    mol_pred = Chem.RenumberAtoms(mol_pred, m_order)
    mol_pred = Chem.RemoveHs(mol_pred)
    mol_pred_pos = np.array(mol_pred.GetConformer().GetPositions())

    rmsd = np.sqrt(((true_ligand_pos - mol_pred_pos) ** 2).sum(axis=1).mean(axis=0))
    kabsch = kabsch_RMSD(mol_pred_pos, true_ligand_pos)
    com_dist = compute_RMSD(mol_pred_pos.mean(axis=0), true_ligand_pos.mean(axis=0))
    info.append([pdb, rmsd, com_dist, kabsch])


In [11]:
# custom description function.
def below_threshold(x, threshold=5):
    return 100 * (x < threshold).sum() / len(x)
def custom_description(data):
    t1 = data
    t2 = t1.describe()
    t3 = t1.iloc[:,1:].apply(below_threshold, threshold=2, axis=0).reset_index(name='2A').set_index('index').T
    t31 = t1.iloc[:,1:].apply(below_threshold, threshold=5, axis=0).reset_index(name='5A').set_index('index').T
    t32 = t1.iloc[:,1:].median().reset_index(name='median').set_index('index').T
    t4 = pd.concat([t2, t3, t31, t32]).loc[['mean', '25%', '50%', '75%', '5A', '2A', 'median']]
    t5 = t4.T.reset_index()
    t5[['Methods', 'Metrics']] = t5['index'].str.split('_', 1, expand=True)
    t6 = pd.pivot(t5, values=['mean', 'median', '25%', '50%', '75%', '5A', '2A'], index=['Methods'], columns=['Metrics'])
    t6_col = t6.columns
    t6.columns = t6_col.swaplevel(0, 1)
    t7 = t6[sorted(t6.columns)]
    my_MultiIndex = [
                (    'RMSD',  'mean'),
                (    'RMSD',   '25%'),
                (    'RMSD',  '50%'),
                (    'RMSD',   '75%'),
                (    'RMSD',  '5A'),
                (    'RMSD', '2A'),
                ('COM_DIST',  'mean'),
                ('COM_DIST',   '25%'),
                ('COM_DIST',  '50%'),
                ('COM_DIST',   '75%'),
                ('COM_DIST',  '5A'),
                ('COM_DIST', '2A'),
                (  'KABSCH',  'mean'),
                (  'KABSCH',   'median'),
                ]
    t8 = t7[my_MultiIndex]

    my_MultiIndex_fancy = [
                (    'Ligand RMSD $\downarrow$', ' ', 'mean'),
                (    'Ligand RMSD $\downarrow$', 'Percentiles $\downarrow$', '25%'),
                (    'Ligand RMSD $\downarrow$', 'Percentiles $\downarrow$',  '50%'),
                (    'Ligand RMSD $\downarrow$', 'Percentiles $\downarrow$',   '75%'),
                (    'Ligand RMSD $\downarrow$', r'% Below Threshold $\uparrow$',  '5A'),
                (    'Ligand RMSD $\downarrow$', r'% Below Threshold $\uparrow$', '2A'),
                ('Centroid Distance $\downarrow$', ' ',  'mean'),
                ('Centroid Distance $\downarrow$', 'Percentiles $\downarrow$',   '25%'),
                ('Centroid Distance $\downarrow$', 'Percentiles $\downarrow$',  '50%'),
                ('Centroid Distance $\downarrow$', 'Percentiles $\downarrow$',   '75%'),
                ('Centroid Distance $\downarrow$', r'% Below Threshold $\uparrow$', '5A'),
                ('Centroid Distance $\downarrow$', r'% Below Threshold $\uparrow$', '2A'),
                (  'KABSCH', 'RMSD $\downarrow$',  'mean'),
                (  'KABSCH', 'RMSD $\downarrow$',   'median'),
                ]

    t8.columns = pd.MultiIndex.from_tuples(my_MultiIndex_fancy)
    return t8.round(2)


In [12]:
d = pd.DataFrame(info, columns=['pdb', 'TankBind_RMSD', 'TankBind_COM_DIST', 'TankBind_KABSCH'])
custom_description(d)

Ligand RMSD $\downarrow$                                      \
                                  Percentiles $\downarrow$              
                             mean                      25%  50%   75%   
Methods                                                                 
TankBind                     7.41                     2.41  3.9  7.86   

                                             Centroid Distance $\downarrow$  \
         % Below Threshold $\uparrow$                                         
                                   5A     2A                           mean   
Methods                                                                       
TankBind                        61.16  20.39                           5.46   

                                                                           \
         Percentiles $\downarrow$            % Below Threshold $\uparrow$   
                              25%   50%  75%                           5A   
Methods                                                                     
TankBind                     0.89  1.77  4.1                        78.51   

                           KABSCH         
                RMSD $\downarrow$         
             2A              mean median  
Methods                                   
TankBind  55.37              2.53   1.98